## DAU на минималках

In [2]:
import csv

with open("log1.tsv", "r", encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter='\t', fieldnames="userid;timestamp;action;value;testids".split(';'))
    data = [row for row in reader][1:]

# print(*data, sep='\n')

res = {}
for row in data:
    t = row['timestamp'].split('T')[0]
    
    if t not in res:
        res[t] = {row['userid'],}
    else:
        res[t].add(row['userid'])

for k, v in res.items():
    print(k, len(v))


2022-08-22 2
2022-08-23 1
2022-08-25 3
2022-08-28 1
2022-08-30 1
2022-09-05 1


## Дедупликация на минималках

In [3]:
import csv

unique = set()

with open("log.dsv", "r", encoding='utf-8') as f:
    lines = f.readlines()
    data = []
    for line in lines:
        # Parse
        row = {arg.split('=')[0]:arg.split('=')[1] for arg in line.split()}
        # Check
        # for old_row in data:
        #     if old_row['userid'] == row['userid'] and old_row['timestamp'] == row['timestamp'] and \
        #             old_row['action'] == row['action'] and old_row['value'] == row['value'] and \
        #             old_row['testids'] == row['testids']:
        #         break
        old_len = len(unique)
        unique.add((row['userid'], row['timestamp'], row['action'], row['value'], row['testids']))
        if len(unique) != old_len:
            data.append(row)

print(*data, sep='\n')
print(len(data))


with open("output.tsv", "w", newline="") as f:
    w = csv.DictWriter(f, delimiter='\t',
                       fieldnames="userid;timestamp;action;value;testids".split(';'))
    w.writeheader()
    w.writerows(data)


{'testids': '13534;23346;23464;25661', 'userid': 'user_2', 'value': '0', 'timestamp': '2022-08-22T12:10:00', 'action': 'product'}
{'testids': '13534;23345;23463;25662', 'value': '12370.0', 'userid': 'user_1', 'action': 'checkout', 'timestamp': '2022-08-22T10:53:00'}
{'value': '12370.0', 'timestamp': '2022-08-22T10:23:00', 'userid': 'user_1', 'testids': '13534;23345;23463;25662', 'action': 'cart'}
{'action': 'category', 'value': '0', 'testids': '13534;23345;23463;25662', 'userid': 'user_1', 'timestamp': '2022-08-22T10:07:00'}
{'timestamp': '2022-08-22T12:14:00', 'value': '12370.0', 'action': 'confirmation', 'testids': '13534;23345;23463;25662', 'userid': 'user_1'}
{'value': '0', 'testids': '13534;23346;23464;25661', 'action': 'search', 'timestamp': '2022-08-22T10:45:00', 'userid': 'user_2'}
{'action': 'product', 'value': '0', 'testids': '13534;23346;23464;25661', 'userid': 'user_2', 'timestamp': '2022-08-22T10:17:00'}
{'action': 'category', 'timestamp': '2022-08-22T12:27:00', 'testids':

## Очистка данных на минималках

In [4]:
import csv


def char_digit_split(s):
    head = s.rstrip('0123456789')
    tail = s[len(head):]
    return head, tail


def test(arr):
    global users, unique, total_sales
    user, time, act = arr
    user = int(user.lstrip('userid='))
    time = int(time)

    if act.startswith('check') and (user, time) not in unique:
        users.add(user)
        unique.add((user, time))
        total_sales += 1


with open("user_activity_bad_log.tsv", "r", encoding='utf-8') as f:
    lines = f.readlines()
    
    users = set()
    unique = set()
    total_sales = 0
    
    for i, line in enumerate(lines):
        line = line.strip()
        length = len(line)

        if 5 <= length <= 50:
            test(line.split())
                
        elif length > 50:
            # print(line, length)
            tmp = []
            for word in line.split():
                if word.isdigit():
                    tmp.append(word)
                else:
                    char, digit = char_digit_split(word)
                    tmp.append(char)
                    test(tmp)
                    tmp = [digit]

print(total_sales, len(users))  # 2505(2492)[+13] 434(443)[-9]
print(round(total_sales / len(users), 5))

2638 457
5.77243


## Когортный анализ на минималках

In [5]:
import datetime

cogort = {}
users_registrate = {}

with open("log2.tsv", "r", encoding='utf-8') as f:  # userid - timestamp - action - value - testids
    lines = f.readlines()
    data = []
    for i, line in enumerate(lines[1:]):
        user_id, date_event, action, value, *_ = line.split()
        date_event = date_event.split('T')[0]  # yyyy-mm-dd
        value = float(value)
        
        # if i % 500 == 0:
        #     print(user_id, date_log, action, value)

        if user_id not in users_registrate:
            user_date = datetime.date.fromisoformat(date_event)
            # cogort_date = user_date.strftime("%Y-%M")
            cogort_date = date_event[:-3]
            users_registrate[user_id] = (user_date, cogort_date)
            if cogort_date not in cogort:
                cogort[cogort_date] = [0, 0, 0, 0]
            
        if action == 'checkout':
            date_1, cogort_date = users_registrate[user_id]
            date_2 = datetime.date.fromisoformat(date_event)
            
            week = (date_2 - date_1).days // 7
            if week < 4:
                cogort[cogort_date][week] += value
            
# print(*cogort.items(), sep='\n')
# print(len(users_registrate))

for cogort_date, value in cogort.items():
    # value = [str(round(val / 1e6)) for val in list(value.values())[:4]]
    value = [str(round(val / 1e6)) for val in value]
    print('\t'.join([cogort_date, '|', *value]))


2022-02	|	72	33	25	19
2022-03	|	29	16	13	13
2022-04	|	23	10	10	9
2022-05	|	21	8	8	7
2022-06	|	16	5	5	5
2022-07	|	15	2	2	0


## Сессионизация на минималках

In [14]:
import datetime
import csv

users_session = {}
users_session_last_time = {}
next_id = 1

with open("log3.tsv", "r", encoding='utf-8') as f:  
    lines = f.readlines()
    data = []
    for i, line in enumerate(lines[1:]):
        user_id, date_iso, action, value, *test_ids = line.split()
        date_event = datetime.datetime.fromisoformat(date_iso)
        
        if user_id not in users_session:
            users_session[user_id] = next_id
            users_session_last_time[user_id] = date_event
            next_id += 1
        elif date_event - users_session_last_time[user_id] > datetime.timedelta(minutes=30):
            users_session[user_id] = next_id
            next_id += 1
        
        users_session_last_time[user_id] = date_event
        data.append([user_id, date_iso, action, value, test_ids, users_session[user_id]])
        
        print(user_id, date_event, action, value, date_event)
        print(date_event - users_session_last_time[user_id], datetime.timedelta(minutes=30),
              date_event, users_session_last_time[user_id], sep='\n')
            
# userid - timestamp - action - value - testids - (sessionid)
with open("output.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter='\t',)
    w.writerow("userid timestamp action value testids sessionid".split())
    w.writerows(data)


user_1 2022-08-22 10:00:00 product 0 2022-08-22 10:00:00
0:00:00
0:30:00
2022-08-22 10:00:00
2022-08-22 10:00:00
user_2 2022-08-22 10:04:00 mainpage 0 2022-08-22 10:04:00
0:00:00
0:30:00
2022-08-22 10:04:00
2022-08-22 10:04:00
user_1 2022-08-22 10:07:00 category 0 2022-08-22 10:07:00
0:00:00
0:30:00
2022-08-22 10:07:00
2022-08-22 10:07:00
user_2 2022-08-22 10:17:00 product 0 2022-08-22 10:17:00
0:00:00
0:30:00
2022-08-22 10:17:00
2022-08-22 10:17:00
user_1 2022-08-22 10:23:00 cart 12370.0 2022-08-22 10:23:00
0:00:00
0:30:00
2022-08-22 10:23:00
2022-08-22 10:23:00
user_2 2022-08-22 10:45:00 search 0 2022-08-22 10:45:00
0:00:00
0:30:00
2022-08-22 10:45:00
2022-08-22 10:45:00
user_1 2022-08-22 10:53:00 checkout 12370.0 2022-08-22 10:53:00
0:00:00
0:30:00
2022-08-22 10:53:00
2022-08-22 10:53:00
user_2 2022-08-22 12:10:00 product 0 2022-08-22 12:10:00
0:00:00
0:30:00
2022-08-22 12:10:00
2022-08-22 12:10:00
user_1 2022-08-22 12:14:00 confirmation 12370.0 2022-08-22 12:14:00
0:00:00
0:30:00
2

## Пиковая нагрузка на минималках

In [27]:
import datetime

sessions = []
session_start_stop = {}
count_online = {}

with open('log4.tsv', 'r', encoding='utf-8') as f:  
    lines = f.readlines()
    data = []
    for i, line in enumerate(lines[1:]):
        user_id, date_iso, *_, session_id = line.split()
        date_event = datetime.datetime.fromisoformat(date_iso)
        # print(user_id, date_event, session_id, sep='\t')

        if session_id not in session_start_stop:
            session_start_stop[session_id] = [date_event, None]
        session_start_stop[session_id][-1] = date_event
            
for key in session_start_stop.keys():
    session_start_stop[key][-1] += datetime.timedelta(minutes=30)
    sessions.append((session_start_stop[key][0], 1))
    sessions.append((session_start_stop[key][-1], -1))
    
sessions.sort(key=lambda x: (x[0], x[1]))

current_count = 0
for session in sessions:
    current_count += session[1]
    count_online[datetime.datetime.isoformat(session[0])] = current_count

max_count = max(count_online.values())
session_times = list(count_online.keys())
for i, session_time in enumerate(session_times):
    if count_online[session_time] == max_count:
        print(session_time, session_times[i+1])
        
# print(*session_start_stop.items(), sep='\n')
# print(*sessions, sep='\n')
# print(*count_online.items(), sep='\n')

user_1	2022-08-22 10:00:00	1
user_2	2022-08-22 10:04:00	3
user_1	2022-08-22 10:07:00	1
user_2	2022-08-22 10:17:00	3
user_1	2022-08-22 10:23:00	1
user_2	2022-08-22 10:45:00	3
user_1	2022-08-22 10:53:00	1
user_2	2022-08-22 12:10:00	4
user_1	2022-08-22 12:14:00	2
user_2	2022-08-22 12:27:00	4
2022-08-22T10:04:00 2022-08-22T11:15:00
2022-08-22T12:14:00 2022-08-22T12:44:00
('1', [datetime.datetime(2022, 8, 22, 10, 0), datetime.datetime(2022, 8, 22, 11, 23)])
('3', [datetime.datetime(2022, 8, 22, 10, 4), datetime.datetime(2022, 8, 22, 11, 15)])
('4', [datetime.datetime(2022, 8, 22, 12, 10), datetime.datetime(2022, 8, 22, 12, 57)])
('2', [datetime.datetime(2022, 8, 22, 12, 14), datetime.datetime(2022, 8, 22, 12, 44)])
(datetime.datetime(2022, 8, 22, 10, 0), 1)
(datetime.datetime(2022, 8, 22, 10, 4), 1)
(datetime.datetime(2022, 8, 22, 11, 15), -1)
(datetime.datetime(2022, 8, 22, 11, 23), -1)
(datetime.datetime(2022, 8, 22, 12, 10), 1)
(datetime.datetime(2022, 8, 22, 12, 14), 1)
(datetime.dateti